# Статистика по продуктам

In [1]:
import matplotlib.pyplot as plt
import psycopg2 as psy
import pandas as pd

from datetime import datetime as dt

def state_flag(row):
    if row['shipped_date'] == None:
        return 1
    else:
        return 0

def sql_to_pd(query, insert_state_flag=False):
    '''
    Функция стучится в базу данных и обрабатывает указанный в аргументе запрос.
    Затем преобразует полученные данные в датафрейм и возвращает его.

    query - SQL запрос в базу данных, которые указанны в функции.
    
    insert_state_flag - подходит исключительно для БД из задания.
                        Если активен, добавляет столбец со статусом
                        заказа в датафрейм используя функцию state_flag()
                        (0-неактивный, 1-активный)
    '''
    try:
        # Connect to an existing database
        connection = psy.connect(user="postgres",
                                  password="Pga1708844!",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="postgres")
        # Create a cursor to perform database operations
        cursor = connection.cursor()
        cursor.execute(query)
        colnames = [desc[0] for desc in cursor.description]
        record = cursor.fetchall()
        # Create a dataframe with the data from a database
        data_frame = pd.DataFrame(record, columns=colnames)
        # Использую функцию выше в случае если нужно поставить флаг статуса заказа, включен по умолчанию
        if insert_state_flag:
            data_frame['order_state'] = data_frame.apply(state_flag, axis=1)
        return data_frame
    except Exception as e:
        print("Error while connecting to PostgreSQL", e)
    finally:
        if (connection):
            cursor.close()
            connection.close()

## 1. Выведите клиента с наибольшим количеством активных заказовМетод to_excel)

In [12]:
df = sql_to_pd('SELECT * FROM orders\
                    INNER JOIN order_details USING(order_id)\
                    INNER JOIN customers USING(customer_id)'
               , insert_state_flag=True
              )
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155 entries, 0 to 2154
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       2155 non-null   object 
 1   order_id          2155 non-null   int64  
 2   employee_id       2155 non-null   int64  
 3   order_date        2155 non-null   object 
 4   required_date     2155 non-null   object 
 5   shipped_date      2082 non-null   object 
 6   ship_via          2155 non-null   int64  
 7   freight           2155 non-null   float64
 8   ship_name         2155 non-null   object 
 9   ship_address      2155 non-null   object 
 10  ship_city         2155 non-null   object 
 11  ship_region       856 non-null    object 
 12  ship_postal_code  2100 non-null   object 
 13  ship_country      2155 non-null   object 
 14  product_id        2155 non-null   int64  
 15  unit_price        2155 non-null   float64
 16  quantity          2155 non-null   int64  


In [17]:
highest_active = df\
                .groupby('customer_id')\
                .agg({'company_name': 'first', 'contact_name': 'first', 'address': 'first', 'order_state':'sum'})\
                .sort_values(by='order_state', ascending=False)\
                .reset_index()
highest_active.rename(columns={'order_state': 'active_orders_count'}, inplace=True)
highest_active.head(1)

,customer_id,company_name,contact_name,address,active_orders_count
0,RATTC,Rattlesnake Canyon Grocery,Paula Wilson,2817 Milton Dr.,25


## 2. Выведите клиентов по количеству активных и неактивных заказов

In [19]:
non_active_orders = df[df['order_state'] == 0]
non_active_orders

,customer_id,order_id,employee_id,order_date,required_date,shipped_date,ship_via,freight,ship_name,ship_address,...,contact_name,contact_title,address,city,region,postal_code,country,phone,fax,order_state
0,VINET,10248,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,...,Paul Henriot,Accounting Manager,59 rue de l'Abbaye,Reims,None,51100,France,26.47.15.10,26.47.15.11,0
1,VINET,10248,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,...,Paul Henriot,Accounting Manager,59 rue de l'Abbaye,Reims,None,51100,France,26.47.15.10,26.47.15.11,0
2,VINET,10248,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,...,Paul Henriot,Accounting Manager,59 rue de l'Abbaye,Reims,None,51100,France,26.47.15.10,26.47.15.11,0
3,TOMSP,10249,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,...,Karin Josephs,Marketing Manager,Luisenstr. 48,Münster,None,44087,Germany,0251-031259,0251-035695,0
4,TOMSP,10249,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,...,Karin Josephs,Marketing Manager,Luisenstr. 48,Münster,None,44087,Germany,0251-031259,0251-035695,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,WHITC,11066,7,1998-05-01,1998-05-29,1998-05-04,2,44.72,White Clover Markets,1029 - 12th Ave. S.,...,Karl Jablonski,Owner,305 - 14th Ave. S. Suite 3B,Seattle,WA,98128,USA,(206) 555-4112,(206) 555-4115,0
2104,WHITC,11066,7,1998-05-01,1998-05-29,1998-05-04,2,44.72,White Clover Markets,1029 - 12th Ave. S.,...,Karl Jablonski,Owner,305 - 14th Ave. S. Suite 3B,Seattle,WA,98128,USA,(206) 555-4112,(206) 555-4115,0
2105,WHITC,11066,7,1998-05-01,1998-05-29,1998-05-04,2,44.72,White Clover Markets,1029 - 12th Ave. S.,...,Karl Jablonski,Owner,305 - 14th Ave. S. Suite 3B,Seattle,WA,98128,USA,(206) 555-4112,(206) 555-4115,0
2106,DRACD,11067,1,1998-05-04,1998-05-18,1998-05-06,2,7.98,Drachenblut Delikatessen,Walserweg 21,...,Sven Ottlieb,Order Administrator,Walserweg 21,Aachen,None,52066,Germany,0241-039123,0241-059428,0


## 3. Нарисуйте круговую диаграмму по топ 10 самых больших по заказам клиентов, разделенных на активные и неактивные заказы

## 4. Выведите продукт с наибольшим количеством активных зказов

## 5. Выведите количества продуктов разделенные по активности

## 6. Нарисуйте диаграмму по топ 10 самым продаваемым продуктам

## 7. Добавьте в вашу таблицу должность и имя сотрудника, а также его город, гдеон работает

## 8. Теперь выведите количество активных и неактивных заказов по сотрудникам

## 9. Выгрузите таблицу с заказами и сотрудниками в эксель. (Метод to_excel)